# **Pretrained Language Models - BERT and RoBERTa**

In [ ]:
# cd twitter16

In [5]:
#Install simpletransformers lib
!pip install simpletransformers

In [1]:
!nvidia-smi

Fri Nov 26 07:57:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100S-PCI...  Off  | 00000000:43:00.0 Off |                    0 |
| N/A   41C    P0    40W / 250W |   3349MiB / 32510MiB |     24%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100S-PCI...  Off  | 00000000:47:00.0 Off |                    0 |
| N/A   

In [2]:
# from google.colab import drive
# drive.mount('gdrive')

In [3]:
# cd /content/gdrive/MyDrive/College/Semester5/NLP/project

In [8]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import pandas as pd
import numpy as np

[nltk_data] Downloading package stopwords to /data/shizhe/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
df_train = pd.read_csv("train.csv")
df_train['author'] = df_train['author'].replace(np.nan, "<NO_AUTHOR>")
df_train['title'] =  df_train['title'].replace(np.nan, "<NO_TITLE>")
df_train['text'] =   df_train['text'].replace(np.nan, "<NO_TEXT>")
for i, j in df_train.iterrows():
    df_train.at[i, 'text'] = j['author']+" | "+j['title'] +" | "+ j['text']
def punctuation_removal(text):
    all_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(all_list)
    return clean_str
df_train['text'] = df_train['text'].apply(punctuation_removal)
stop = stopwords.words('english')
df_train['text'] = df_train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_train = df_train.sample(frac=1)

In [10]:
import pandas as pd
import pickle
import numpy as np

In [15]:
df_train

,id,title,author,text,label
4876,4876,WATCH: ‘Damn!’ Liberal Hosts Flustered Over NE...,Shae Weatherall,Shae Weatherall WATCH ‘Damn’ Liberal Hosts Flu...,1
20417,20417,"As Anger at O’Reilly Builds, Activists Use Soc...",Michael M. Grynbaum and Sapna Maheshwari,Michael M Grynbaum Sapna Maheshwari As Anger O...,0
15584,15584,"Syria, Hungary, Marine Le Pen: Your Wednesday ...",Patrick Boehler,Patrick Boehler Syria Hungary Marine Le Pen Yo...,0
18044,18044,Hillary and the Ghosts of Watergate,Charles Hugh Smith,Charles Hugh Smith Hillary Ghosts Watergate Po...,1
2998,2998,Oregon Man Accused of Trying to Shove Co-Worke...,Katherine Rodriguez,Katherine Rodriguez Oregon Man Accused Trying ...,0
...,...,...,...,...,...
7988,7988,Chart Of The Day: The Explosion Of Student Loa...,David Stockman,David Stockman Chart Of The Day The Explosion ...,1
3130,3130,FS1’s Bayless: Athletes Skipping White House B...,Trent Baker,Trent Baker FS1’s Bayless Athletes Skipping Wh...,0
746,746,Breitbart News Daily: Draining the Swamp - Bre...,Breitbart News,Breitbart News Breitbart News Daily Draining S...,0
3288,3288,Iceland’s Pirate Party To Make History And Win...,IWB,IWB Iceland’s Pirate Party To Make History And...,1


In [17]:
from sklearn.model_selection import train_test_split
# file = open("split.pkl", 'rb')
train_x, valid_x, train_y, valid_y = train_test_split(df_train['text'], df_train['label'], test_size = 0.2, random_state=42)

In [18]:
train_x

5271     Nick Corasaniti Maggie Haberman Donald Trump S...
2601     Charles McDermid Hillary Clinton Donald Trump ...
8238     Dean Baker Inequality Policy Globalization tec...
7502     Katherine Rodriguez Police Man Finds Nude Phot...
17218    Pakalert 10 Ways Russia Preparing For World Wa...
                               ...                        
12714    Elina StOnge Trump Winning Why It May Have Bee...
13220    Edwin Mora Report More Citizens Saudi Arabia H...
17843    MeThePeople Comment Friday funnies MeThePeople...
17063    Alan Rappeport Alexander Burns Highlights Hill...
12215    Aaron Klein SorosFinanced Groups Provided Scri...
Name: text, Length: 16640, dtype: object

In [20]:
df_train = pd.DataFrame()
df_train['text'] = np.array(train_x)
df_train['labels'] = np.array(train_y)
df_test = pd.DataFrame()
df_test['text'] = np.array(valid_x)
df_test['labels'] = np.array(valid_y)

In [21]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd

model_args = ClassificationArgs(overwrite_output_dir= True, num_train_epochs=2)

model = ClassificationModel( "bert", 'bert-base-cased', args=model_args)

# df_train = df_train.rename(columns={'label':'labels'})#read training data file
df_train = df_train.sample(frac = 1)
train_df = df_train
eval_df = df_test

model.train_model(train_df) #Start model training

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/16640 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/2080 [00:00<?, ?it/s]

/data/shizhe/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 2:   0%|          | 0/2080 [00:00<?, ?it/s]

(4160, 0.05275420834883474)

In [22]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

  0%|          | 0/4160 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/520 [00:00<?, ?it/s]

In [23]:
p = result['tp']/(result['tp']+result['fp'])
print("Precision", p)

Precision 0.998133457769482


In [24]:
n = result['tp']/(result['tp']+result['fn'])
print("Recall",n )

Recall 0.9972027972027973


In [25]:
print("F1", 2*p*n/(p+n))

F1 0.9976679104477612


In [26]:
import numpy as np
print("Accuracy", sum((np.array([np.argmax(i) for i in model_outputs])==eval_df['labels']).astype(int))/len(eval_df))

Accuracy 0.9975961538461539


In [27]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd

model_args = ClassificationArgs(overwrite_output_dir= True, num_train_epochs=2)

model = ClassificationModel( "roberta", 'roberta-base', args=model_args)

# df_train = df_train.rename(columns={'label':'labels'})#read training data file
df_train = df_train.sample(frac = 1)
train_df = df_train
eval_df = df_test

model.train_model(train_df) #Start model training

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

  0%|          | 0/16640 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/2080 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/2080 [00:00<?, ?it/s]

(4160, 0.06572537547192321)

In [28]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
p = result['tp']/(result['tp']+result['fp'])
print("Precision", p)
n = result['tp']/(result['tp']+result['fn'])
print("Recall",n )
print("F1", 2*p*n/(p+n))
import numpy as np
print("Accuracy", sum((np.array([np.argmax(i) for i in model_outputs])==eval_df['labels']).astype(int))/len(eval_df))

  0%|          | 0/4160 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/520 [00:00<?, ?it/s]

Precision 0.9981299672744273
Recall 0.9953379953379954
F1 0.9967320261437909
Accuracy 0.9966346153846154
